<a href="https://colab.research.google.com/github/Payal1809/Machine-Learning-Projects-/blob/main/DeepAb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepAb antibody structure prediction notebook

Official notebook for [DeepAb](https://www.sciencedirect.com/science/article/pii/S2666389921002804): Antibody structure prediction using interpretable deep learning. This notebook is made freely available for non-commercial use. Commercial use is available under the terms of the [Rosetta-DL license](https://github.com/RosettaCommons/Rosetta-DL).

For more details about DeepAb, please read:
[Ruffolo, J. A., Sulam, J., & Gray, J. J. (2022). Antibody structure prediction using interpretable deep learning. _Patterns_, 3(2), 100406.](https://www.sciencedirect.com/science/article/pii/S2666389921002804)

In [1]:
#@title Input antibody Fv sequences then press `Runtime` -> `Run all`

#@markdown To predict single-chain antibodies (nanobodies) or individual light chains, just omit the other chain sequence.

import os

name = "my_antibody" #@param {type:"string"}
pred_dir = name
os.makedirs(pred_dir, exist_ok=True)

heavy_sequence = 'EIQLQQSGPELVKPGASVKISCKASGYSFTDYIMLWVKQSHGKSLEWIGNINPYYGSTSYNLKFKGKATLTVDKSSSTAYMQLNSLTSEDSAVYYCARKNYYGSSLDYWGQGTTLTVS' #@param {type:"string"}
light_sequence = 'DVVMTQTPFSLPVSLGDQASISCRSSQSLVHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQSTHVPYTFGGGTKLEIK' #@param {type:"string"}

decoys = 1 #@param {type:"integer"}
single_chain = len(heavy_sequence) == 0 or len(light_sequence) == 0
renumber = True #@param {type:"boolean"}

In [5]:
#@title Install DeepAb code and dependencies

#@markdown This step will clone the DeepAb repository and install the required dependencies.
#@markdown Be sure to obtain the appropriate [PyRosetta license](https://www.pyrosetta.org/home/licensing-pyrosetta). This step may request access to your Google Drive and require restarting your runtime. If this is your first time running the notebook, this step may take several minutes.

! git clone http://github.com/rosettacommons/deepab
# Modify requirements.txt to remove torch and torchvision version specifiers
! sed -i 's/torch==1.7.1/torch/' deepab/requirements.txt
! sed -i 's/torchvision==0.8.2/torchvision/' deepab/requirements.txt
%pip install -r deepab/requirements.txt

import sys
from glob import glob
from tqdm.contrib.concurrent import process_map
import torch
import numpy as np

sys.path.insert(0, "deepab")

OUT_DIR = "predict_structure"
os.makedirs(OUT_DIR, exist_ok=True)

# Notebook setup
if 'google.colab' in sys.modules:
  !pip install pyrosettacolabsetup
  import pyrosettacolabsetup
  if not os.path.exists("/content/google_drive/MyDrive/PyRosetta/colab.bin"):
    pyrosettacolabsetup.install_pyrosetta()
  else:
    pyrosettacolabsetup.mount_pyrosetta_install()

import pyrosetta
pyrosetta.init(
    "-mute all -detect_disulf true -detect_disulf_tolerance 1.5 -check_cdr_chainbreaks false"
)

from predict import load_model, ModelEnsemble, get_cst_defs, build_structure, renumber_pdb
from annotate_attention import cdr_indices, annotate_structure

fatal: destination path 'deepab' already exists and is not an empty directory.


MessageError: Error: credential propagation was unsuccessful

In [ ]:
#@title Download trained models

#@markdown This step may take several minutes if models are not found locally.

model_files = [
    "ensemble_abresnet/rs0.pt",
    "ensemble_abresnet/rs1.pt",
    "ensemble_abresnet/rs2.pt",
    "ensemble_abresnet/rs3.pt",
    "ensemble_abresnet/rs4.pt",
]

do_download = all([not os.path.exists(f) for f in model_files])
if do_download:
    os.system("wget https://data.graylab.jhu.edu/ensemble_abresnet_v1.tar.gz")
    os.system("tar -xf ensemble_abresnet_v1.tar.gz")

device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)
model = ModelEnsemble(model_files=model_files,
                          load_model=load_model,
                          eval_mode=True,
                          device=device)
single_model = load_model(model_files[0],
                          eval_mode=True,
                          device=device)

In [ ]:
#@title Predict antibody structure with DeepAb

#@markdown Antibody structure prediction with DeepAb involves three main steps: constraint generation, initial structure prediction by multi-dimensional scaling, and structural refinement with PyRosetta.
#@markdown Expected runtime is approximately ten minutes for five decoys.

fasta_file = os.path.join(pred_dir, f"{name}.fasta")
with open(fasta_file, "w") as f:
    f.write(">:H\n{}\n>:L\n{}\n".format(heavy_sequence, light_sequence))
cst_defs = get_cst_defs(model, fasta_file, device=device)

pred_pdb = build_structure(model,
                                fasta_file,
                                cst_defs,
                                pred_dir,
                                target=name,
                                num_decoys=decoys,
                                num_procs=decoys,
                                single_chain=single_chain,
                                device=device)

if renumber:
    renumber_pdb(pred_pdb, pred_pdb)

In [ ]:
#@title Annotate predicted structure with output attention.

#@markdown Visualize the inter-residue attention from an output geometry branch from DeepAb. The attention scores will replace the B-factors in the predicted structure.

cdr_loop = "h3" #@param ["h1", "h2", "h3", "l1", "l2", "l3"]
cdr_i = cdr_indices(pred_pdb, cdr_loop)

attn_branch = "ca" #@param ["ca", "cb", "no", "omega", "theta", "phi"]

attn_pdb_file = pred_pdb.replace(".pdb", f".{cdr_loop}_attn.pdb")
os.system(f"cp {pred_pdb} {attn_pdb_file}")

annotate_structure(single_model, fasta_file, attn_pdb_file, cdr_i, attn_branch)

In [ ]:
from google.colab import files

#@title Download results

#@markdown Download zip file containing structure prediction and annotation results. If download fails, results are also accessible from file explorer on the left panel of the notebook.

!zip -FSr $name".result.zip" $pred_dir/
files.download(f"{name}.result.zip")